In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import numpy as np

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [2]:
import pandas as pd

dfEmbeddings = pd.read_csv(
    'C:/Users/alire/OneDrive/data/statman_bitbucket/aki/LLM/March2024/openai_3large_operation.csv'
)
dfEmbeddings.head()

,project_id,operation_no,X1,X2,X3,X4,X5,X6,X7,X8,...,X3063,X3064,X3065,X3066,X3067,X3068,X3069,X3070,X3071,X3072
0,PR-00000001,1,0.011898,0.036309,-0.005463,0.008832,0.014760,0.027967,-0.035566,0.017884,...,-0.010715,0.010800,-0.012413,0.012261,0.008452,0.008917,0.010977,-0.029892,-0.010766,-0.010437
1,PR-00000002,2,0.000332,0.027386,-0.020735,-0.030093,0.011078,0.034497,-0.051592,-0.001025,...,-0.004109,0.002861,0.003123,-0.000474,0.001826,0.023946,0.022539,-0.020903,-0.003339,0.002045
2,PR-00000003,3,-0.013588,0.006824,-0.002654,0.027271,0.033145,0.019402,-0.031663,0.027783,...,0.004871,0.006225,-0.000025,0.002941,-0.011971,0.019294,0.002917,0.003523,0.004716,0.002019
3,PR-00000004,4,0.028444,0.027613,-0.017272,-0.006343,0.022245,0.052718,-0.014736,0.047054,...,0.011954,-0.006590,0.000384,-0.010179,0.002074,0.015004,0.001219,-0.010087,0.011813,0.037503
4,PR-00000005,5,0.015713,0.057943,-0.016102,0.009510,-0.026413,0.027086,-0.050581,0.033581,...,-0.001399,-0.003857,-0.012810,-0.008986,-0.001552,0.009922,0.005754,-0.013910,-0.013042,-0.006614


In [3]:
from TextEmbeddingFE.main import cluster_embeddings#, gener
my_labels = cluster_embeddings(
    dfEmbeddings.iloc[:, 2:]
    , n_clusters = 10
    , n_init = 10
)

In [4]:
dfText = pd.read_csv(
    'C:/Users/alire/OneDrive/data/statman_bitbucket/aki/LLM/all_text_columns_na_handled.csv'
)
dfText.head()

,project_id,operation_no,2.01 Diagnosis,2.02 Previous Procedure,2.07 Comorbid Conditions,3.09 Operation performed
0,PR-00000001,1,155500. Cardiac conduit complication;010125. P...,123610. Replacement of cardiac conduit;123601....,140206. 22q11 microdeletion with full DiGeorge...,123610. Replacement of cardiac conduit
1,PR-00000002,2,155516. Cardiac conduit failure;103604. Aortic...,No previous procedure,102000. No pre-procedural risk factors,123610. Replacement of cardiac conduit;121614....
2,PR-00000003,3,010101. Tetralogy of Fallot;090591. Pulmonary ...,No previous procedure,102000. No pre-procedural risk factors,122621. Absent pulmonary valve syndrome (Fallo...
3,PR-00000004,4,091591. Aortic regurgitation;091519. Congenita...,123601. RV to pulmonary artery conduit constru...,102000. No pre-procedural risk factors,121791. Aortic root replacement: valve sparing...
4,PR-00000005,5,155516. Cardiac conduit failure;090101. Common...,123601. RV to pulmonary artery conduit constru...,160305. Lung disease;158210. Renal failure;140...,123610. Replacement of cardiac conduit;123452....


In [22]:
def interpret_clusters(
    openai_client
    , prompt
    , openai_textgen_model = 'gpt-4-turbo'
    , temperature = 1.0
):
    response = openai_client.chat.completions.create(
        model = openai_textgen_model
        #, response_format = {"type": "json_object"}
        , messages = [
            {"role": "user", "content": prompt}
        ]
        , temperature = temperature
    )
    return response.choices[0].message.content

In [15]:
from TextEmbeddingFE.main import generate_prompt

my_own_preamble = (
    "The following represents a group of pediatric patients undergoing cardiopulmonary bypass."
    " Each row contains one or more surgical procedures performed on the patient during bypass, separated by ';'."
    " Patients are grouped into 10 groups according to the similarity of their surgical procedures."
    " Please suggest group labels that are representative of their members, and also distinct from each other:"
)

ntokens, my_prompt = generate_prompt(
    text_list = list(dfText['3.09 Operation performed'])#[:5]
    , cluster_labels = my_labels#[:5]
    , prompt_observations = 'patients'
    , prompt_texts = 'surgical operations'
    , preamble = my_own_preamble
)
ntokens

25323

In [23]:
my_out = interpret_clusters(
    openai_client = client
    , prompt = my_prompt
)

In [27]:
my_out

ChatCompletion(id='chatcmpl-9JD5LAhi6L00zzNjAGGzzvnnYpokV', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Based on the surgical procedures listed for each group, here is a set of suggested labels for the 10 groups, which are descriptive of the key characteristics:\n\n1. **Group 1: Heart and Lung Transplant Group**\n   - Procedures: Heart transplantation and double lung transplant.\n\n2. **Group 2: Septal Defects and Complementary Cardiac Procedures Group**\n   - Procedures: Atrial septal defect closure, VSD closure, pacemaker placement, and additional cardiac repairs.\n\n3. **Group 3: Complex Congenital Heart Defect Repair Group**\n   - Procedures: Repairs associated with conditions like Tetralogy of Fallot, absent pulmonary valve syndrome, and double outlet right ventricle.\n\n4. **Group 4: Comprehensive Cardiac Repair and Arterioplasty Group**\n   - Procedures: Wide range of complex repairs including valve repairs/replacements, a